In [1]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData

stockdata = GetStockData() 
spark = DevSparkSession().spark

from pyspark.sql.functions import col,explode,sha2,concat_ws,trim, lit


## create table 
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table


spark.sql( f"""
    CREATE TABLE delta_table (
        Symbol STRING,
        ExchangeName STRING,
        Currency STRING,
        Type STRING,
        ExchangeTimeZone STRING,
        Volume STRING,
        High STRING,
        Low STRING,
        Close STRING,
        Open STRING,
        Date STRING,
        __CurrentFlag BOOLEAN,
        __DeletedFlag BOOLEAN,
        __EffectiveStartDateTime TIMESTAMP,
        __EffectiveEndDateTime TIMESTAMP,
        __lastmodified TIMESTAMP,
        __HashKey STRING,
        __HashValue STRING
)
USING DELTA
LOCATION  '{silver_path}';          
        """)


# ## re-create table 


# truncate_schema = spark.read.format("delta").load(silver_path).schema

# truncate_silver = spark.createDataFrame([], schema= truncate_schema)
# truncate_silver.write.format("delta").mode("overwrite").save(silver_path)



ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-70b1a4c2-243c-46e4-812c-71f6874f0fd0;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 88ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

DataFrame[]

### Bronze -> Silver

In [ ]:

# from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType


# schema = StructType([StructField('Symbol', StringType(), True), 
#             StructField('ExchangeName', StringType(), True), 
#             StructField('Currency', StringType(), True), 
#             StructField('Type', StringType(), True), 
#             StructField('ExchangeTimeZone', StringType(), True), 
#             StructField('Volume', StringType(), True), 
#             StructField('High', StringType(), True), 
#             StructField('Low', StringType(), True), 
#             StructField('Close', StringType(), True), 
#             StructField('Open', StringType(), True), 
#             StructField('Date', StringType(), True)])

df_silver = spark.createDataFrame( df_bronze , schema = schema )

df_silver.show()